In [1]:
#%%capture
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.5.0/spark-sql-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.5.0/spark-streaming-kafka-0-10_2.12-3.5.0.jar"
#!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.5.0/spark-avro_2.12-3.5.0.jar"

In [2]:
#!pip install confluent_kafka

In [40]:
import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0,org.apache.spark:spark-avro_2.12:3.5.0 pyspark-shell' 

In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark.sql.functions as func

In [42]:
from confluent_kafka.schema_registry import SchemaRegistryClient
from pyspark.sql.avro.functions import from_avro, to_avro
spark = SparkSession.builder.appName("readFromKafka").master("local").getOrCreate()

In [43]:
def get_schema_from_schema_registry(schema_registry_url, schema_registry_subject):
    sr = SchemaRegistryClient({'url': schema_registry_url})
    latest_version = sr.get_latest_version(schema_registry_subject)
    
    return sr, latest_version

In [44]:
kafka_url = "broker:29092"
kafka_topic_input = "quickstart-jdbc-test"
kafka_topic_output = "quickstart-clean"
schema_registry_url = "http://schema-registry:8081"
schema_registry_subject = f"{kafka_topic_input}-value"
# get schema using subject name
_, avro_version_ = get_schema_from_schema_registry(schema_registry_url, schema_registry_subject)

fromAvroOptions = {"mode":"PERMISSIVE"}
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_url) \
  .option("subscribe", kafka_topic_input) \
  .option("startingOffsets", "earliest") \
  .load() \
  .selectExpr("substring(value, 6) as avro_value") \
  .select(from_avro(col("avro_value"), avro_version_.schema.schema_str, fromAvroOptions).alias("data"))

df.printSchema()
# Skip the first 5 bytes (reserved by schema registry encoding protocol)
#solution I found on Internet https://stackoverflow.com/a/65818275

root
 |-- data: struct (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- email: string (nullable = true)
 |    |-- department: string (nullable = true)
 |    |-- modified: timestamp (nullable = true)



In [45]:
df = df.select(to_avro(func.struct(
        func.col("data.id"),
        func.col("data.name"),
        func.col("data.email"),
        func.col("data.department"),
        func.col("data.modified")
    ), avro_version_.schema.schema_str).alias("value")).select("value")

In [46]:
df.printSchema()

root
 |-- value: binary (nullable = false)



In [48]:
query = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_url) \
    .option("topic", kafka_topic_output) \
    .option("checkpointLocation", "checkpoint") \
    .start() \
    .awaitTermination()
#query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [36]:
query = df \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 